# US Travels : Business / Pleasure / Education / All
### Data Engineering Capstone Project ( Spark )
#### Preparing queries for Fact & Dimension tables

<div style="background-color:#ffe5e5;">
<br>    
    <b>Important</b> : Initially thought of using this Notebook to prepare queries to INSERT into Fact & Dimension tables and Analytics report, but found out Spark SQL is very slow with 50k rows itself as Spark SQL does full scan of the dataset for every WHERE clause, so quitting the idea of using SPARK Tables.

<div>Below codes were used to write a small sample output of i94 dataset containing 10k rows.</div>
<br>
</div>

##### Import packages

In [2]:
import sys
# Below used by fuzzy matching
!{sys.executable} -m pip install fuzzywuzzy[speedup]

    100% |████████████████████████████████| 51kB 4.6MB/s ta 0:00:011
  Running setup.py bdist_wheel for python-levenshtein ... done
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [3]:
import pandas as pd
import os
import glob
#import psycopg2
import pandas as pd
import numpy as np
#from sql_queries import *
import datetime as dt
import json
from fuzzywuzzy import fuzz, process

In [4]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, FloatType, TimestampType, LongType, DateType, NullType

import datetime #Required for ts conversion
#from pyspark.sql.functions import udf
#from pyspark.sql import functions as F

#import pyspark.sql.functions as F

#from pyspark.sql.functions import lit
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second

#from pyspark.sql.functions import *
from pyspark.sql.functions import udf, lit, datediff, when, col

##### Create spark session with hadoop-aws package

In [4]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [5]:
spark.sparkContext.getConf().getAll()

[('spark.submit.pyFiles',
  '/root/.ivy2/jars/saurfang_spark-sas7bdat-2.0.0-s_2.11.jar,/root/.ivy2/jars/com.epam_parso-2.0.8.jar,/root/.ivy2/jars/org.apache.logging.log4j_log4j-api-scala_2.11-2.7.jar,/root/.ivy2/jars/org.slf4j_slf4j-api-1.7.5.jar,/root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar'),
 ('spark.repl.local.jars',
  'file:///root/.ivy2/jars/saurfang_spark-sas7bdat-2.0.0-s_2.11.jar,file:///root/.ivy2/jars/com.epam_parso-2.0.8.jar,file:///root/.ivy2/jars/org.apache.logging.log4j_log4j-api-scala_2.11-2.7.jar,file:///root/.ivy2/jars/org.slf4j_slf4j-api-1.7.5.jar,file:///root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar'),
 ('spark.executor.id', 'driver'),
 ('spark.jars.packages', 'saurfang:spark-sas7bdat:2.0.0-s_2.11'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.app.id', 'local-1582388996084'),
 ('spark.files',
  'file:///root/.ivy2/jars/saurfang_spark-sas7bdat-2.0.0-s_2.11.jar,file:///root/.ivy2/jars/com.epam_parso-2.0.8.jar,file:///root/.ivy2/jars/org.apache.l

In [6]:
spark

In [7]:
# Panda options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# for all rows set to None
# pd.set_option('display.max_rows', None) 

pd.set_option('display.max_rows', 100) 

##### Function

##### Gather

In [14]:
print("{} : Read csv to dataframes".format(dt.datetime.now()))
dfs_visatype1 = spark.read.csv('outputs/df_visatype1.csv', header=True)
dfs_ac1 = spark.read.csv('outputs/df_ac1.csv', header=True)
dfs_USPoE1 = spark.read.csv('outputs/df_USPoE1.csv', header=True)
dfs_uszips1 = spark.read.csv('outputs/df_uszips1.csv', header=True)
dfs_i94countrycode1 = spark.read.csv('outputs/df_i94countrycode1.csv', header=True)
dfs_uscd1 = spark.read.csv('outputs/df_uscd1.csv', header=True)
dfs_usdp = spark.read.csv('outputs/df_usdp.csv', header=True)
dfs_usdr = spark.read.csv('outputs/df_usdr.csv', header=True)
dfs_temper1 = spark.read.csv('outputs/df_temper1.csv', header=True)
dfs_wc1 = spark.read.csv('outputs/df_wc1.csv', header=True)

dfs_alc = spark.read.csv('outputs/df_alc.csv', header=True)
dfs_visapost = spark.read.csv('outputs/df_visapost.csv', header=True)
dfs_alpha2countrycode = spark.read.csv('outputs/df_alpha2countrycode.csv', header=True)
dfs_iap = spark.read.csv('outputs/df_iap.csv', header=True)
dfs_USstatecode = spark.read.csv('outputs/df_USstatecode.csv', header=True)

2020-02-22 16:34:31.051485 : Read csv to dataframes


In [15]:
i94_schema = StructType([
 StructField("CoC", IntegerType()),
 StructField("CoR", IntegerType()),
 StructField("PoE", StringType()),
 StructField("landing_state", StringType()),
 StructField("age", IntegerType()),
 StructField("visa_issued_in", StringType()),
 StructField("occup", StringType()),
 StructField("biryear", IntegerType()),
 StructField("gender", StringType()),
 StructField("airline", StringType()),
 StructField("admnum", IntegerType()),
 StructField("fltno", StringType()),
 StructField("visatype", StringType()),
 StructField("arrival_mode", StringType()),
 StructField("visit_purpose", StringType()),
 StructField("arrival_dt", DateType()),
 StructField("departure_dt", DateType()),
 StructField("DaysinUS", IntegerType()),
 StructField("added_to_i94", DateType()),
 StructField("allowed_until", DateType()),
 StructField("entry_exit", StringType()),
 StructField("month", IntegerType())
])
print("{} : Read csv gzip to dataframe".format(dt.datetime.now()))

# This limits reading to 50k rows as the whole file contains around 28M rows
dfs_ids1 = spark.read.schema(i94_schema).csv('outputs-gzip/dfs_ids1.gzip', header=True).limit(10000)

2020-02-22 16:34:36.410255 : Read csv gzip to dataframe


In [16]:
dfs_ids1.explain()

== Physical Plan ==
CollectLimit 10000
+- *(1) FileScan csv [CoC#704,CoR#705,PoE#706,landing_state#707,age#708,visa_issued_in#709,occup#710,biryear#711,gender#712,airline#713,admnum#714,fltno#715,visatype#716,arrival_mode#717,visit_purpose#718,arrival_dt#719,departure_dt#720,DaysinUS#721,added_to_i94#722,allowed_until#723,entry_exit#724,month#725] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/workspace/outputs-gzip/dfs_ids1.gzip], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<CoC:int,CoR:int,PoE:string,landing_state:string,age:int,visa_issued_in:string,occup:string...


In [17]:
dfs_ids1.write\
  .format("com.databricks.spark.csv")\
  .option("header","true")\
  .option("codec", "org.apache.hadoop.io.compress.GzipCodec")\
  .save('./outputs-gzip/dfs_ids1_10k.gzip')

In [18]:
dfs_ids1 = spark.read.csv('outputs-gzip/dfs_ids1_10k.gzip', header=True)

In [19]:
%%time
#dfs_ids1.count()
dfs_ids1.show()

+---+---+---+-------------+---+--------------+-----+-------+------+-------+----------+-----+--------+------------+-------------+----------+------------+--------+------------+-------------+----------+-----+
|CoC|CoR|PoE|landing_state|age|visa_issued_in|occup|biryear|gender|airline|    admnum|fltno|visatype|arrival_mode|visit_purpose|arrival_dt|departure_dt|DaysinUS|added_to_i94|allowed_until|entry_exit|month|
+---+---+---+-------------+---+--------------+-----+-------+------+-------+----------+-----+--------+------------+-------------+----------+------------+--------+------------+-------------+----------+-----+
|140|140|NYC|           NY| 45|          null| null|   1971|     F|     DL|2147483647|09858|      WT|         Air|     Pleasure|2016-07-01|  2016-07-22|      21|  2016-07-01|   2016-09-28|      exit|    7|
|258|258|HOU|           TX|  5|           KRC| null|   2011|     F|     TK|2147483647|00033|      B2|         Air|     Pleasure|2016-07-10|  2016-08-26|      47|  2016-07-10|  

In [20]:
##### Rows Count ##### 
print("{} : Cleaned dataframes rows count".format(dt.datetime.now()))
print("{:15} = {}".format("dfs_visatype1", dfs_visatype1.count() ))
print("{:15} = {}".format("dfs_ac1", dfs_ac1.count() ))
print("{:15} = {}".format("dfs_USPoE1", dfs_USPoE1.count() ))
print("{:15} = {}".format("dfs_uszips1", dfs_uszips1.count() ))

#Full dataframe is 28217609(process-df.py)
print("{:15} = {}".format("dfs_ids1", dfs_ids1.count()))    

print("{:15} = {}".format("dfs_i94countrycode1", dfs_i94countrycode1.count() ))
print("{:15} = {}".format("dfs_uscd1", dfs_uscd1.count() ))
print("{:15} = {}".format("dfs_usdp", dfs_usdp.count() ))
print("{:15} = {}".format("dfs_usdr", dfs_usdr.count() ))
print("{:15} = {}".format("dfs_temper1", dfs_temper1.count() ))
print("{:15} = {}".format("dfs_wc1", dfs_wc1.count() ))

print("{} : Datasets with no changes/updates".format(dt.datetime.now()))
print("{:15} = {}".format("dfs_alc", dfs_alc.count() ))
print("{:15} = {}".format("dfs_visapost", dfs_visapost.count() ))
print("{:15} = {}".format("dfs_alpha2countrycode", dfs_alpha2countrycode.count() ))
print("{:15} = {}".format("dfs_iap", dfs_iap.count() ))
print("{:15} = {}".format("dfs_USstatecode", dfs_USstatecode.count() ))    

2020-02-22 16:39:48.964385 : Cleaned dataframes rows count
dfs_visatype1   = 108
dfs_ac1         = 113
dfs_USPoE1      = 526
dfs_uszips1     = 33099
dfs_ids1        = 10000
dfs_i94countrycode1 = 236
dfs_uscd1       = 2883
dfs_usdp        = 594
dfs_usdr        = 594
dfs_temper1     = 639649
dfs_wc1         = 15493
2020-02-22 16:39:52.065908 : Datasets with no changes/updates
dfs_alc         = 1571
dfs_visapost    = 284
dfs_alpha2countrycode = 249
dfs_iap         = 1304
dfs_USstatecode = 62


##### Assess

In [21]:
dfs_visatype1.createOrReplaceTempView("stab_visatype1")
dfs_ac1.createOrReplaceTempView("stab_ac1")
dfs_USPoE1.createOrReplaceTempView("stab_USPoE1")
dfs_uszips1.createOrReplaceTempView("stab_uszips1")
dfs_i94countrycode1.createOrReplaceTempView("stab_i94countrycode1")
dfs_uscd1.createOrReplaceTempView("stab_uscd1")
dfs_usdp.createOrReplaceTempView("stab_usdp")
dfs_usdr.createOrReplaceTempView("stab_usdr")
dfs_temper1.createOrReplaceTempView("stab_temper1")
dfs_wc1.createOrReplaceTempView("stab_wc1")

dfs_alc.createOrReplaceTempView("stab_alc")
dfs_visapost.createOrReplaceTempView("stab_visapost")
dfs_alpha2countrycode.createOrReplaceTempView("stab_alpha2countrycode")
dfs_iap.createOrReplaceTempView("stab_iap")
dfs_USstatecode.createOrReplaceTempView("stab_USstatecode")

dfs_ids1.createOrReplaceTempView("stab_ids1")

In [22]:
spark.sql('''
select * from stab_alc limit 2
''').show()

+----+------------------+
|code|       description|
+----+------------------+
| 02Q|     Titan Airways|
| 04Q|Tradewind Aviation|
+----+------------------+



In [23]:
%%time
spark.sql('''
select * from stab_ids1 limit 2
''').show()

+---+---+---+-------------+---+--------------+-----+-------+------+-------+----------+-----+--------+------------+-------------+----------+------------+--------+------------+-------------+----------+-----+
|CoC|CoR|PoE|landing_state|age|visa_issued_in|occup|biryear|gender|airline|    admnum|fltno|visatype|arrival_mode|visit_purpose|arrival_dt|departure_dt|DaysinUS|added_to_i94|allowed_until|entry_exit|month|
+---+---+---+-------------+---+--------------+-----+-------+------+-------+----------+-----+--------+------------+-------------+----------+------------+--------+------------+-------------+----------+-----+
|140|140|NYC|           NY| 45|          null| null|   1971|     F|     DL|2147483647|09858|      WT|         Air|     Pleasure|2016-07-01|  2016-07-22|      21|  2016-07-01|   2016-09-28|      exit|    7|
|258|258|HOU|           TX|  5|           KRC| null|   2011|     F|     TK|2147483647|00033|      B2|         Air|     Pleasure|2016-07-10|  2016-08-26|      47|  2016-07-10|  

In [24]:
%%time
dfs_ids1.take(2)

CPU times: user 5.22 ms, sys: 2.35 ms, total: 7.58 ms
Wall time: 239 ms


[Row(CoC='140', CoR='140', PoE='NYC', landing_state='NY', age='45', visa_issued_in=None, occup=None, biryear='1971', gender='F', airline='DL', admnum='2147483647', fltno='09858', visatype='WT', arrival_mode='Air', visit_purpose='Pleasure', arrival_dt='2016-07-01', departure_dt='2016-07-22', DaysinUS='21', added_to_i94='2016-07-01', allowed_until='2016-09-28', entry_exit='exit', month='7'),
 Row(CoC='258', CoR='258', PoE='HOU', landing_state='TX', age='5', visa_issued_in='KRC', occup=None, biryear='2011', gender='F', airline='TK', admnum='2147483647', fltno='00033', visatype='B2', arrival_mode='Air', visit_purpose='Pleasure', arrival_dt='2016-07-10', departure_dt='2016-08-26', DaysinUS='47', added_to_i94='2016-07-10', allowed_until='2017-01-09', entry_exit='exit', month='7')]

In [25]:
%%time
dfs_ids1.show(2)

+---+---+---+-------------+---+--------------+-----+-------+------+-------+----------+-----+--------+------------+-------------+----------+------------+--------+------------+-------------+----------+-----+
|CoC|CoR|PoE|landing_state|age|visa_issued_in|occup|biryear|gender|airline|    admnum|fltno|visatype|arrival_mode|visit_purpose|arrival_dt|departure_dt|DaysinUS|added_to_i94|allowed_until|entry_exit|month|
+---+---+---+-------------+---+--------------+-----+-------+------+-------+----------+-----+--------+------------+-------------+----------+------------+--------+------------+-------------+----------+-----+
|140|140|NYC|           NY| 45|          null| null|   1971|     F|     DL|2147483647|09858|      WT|         Air|     Pleasure|2016-07-01|  2016-07-22|      21|  2016-07-01|   2016-09-28|      exit|    7|
|258|258|HOU|           TX|  5|           KRC| null|   2011|     F|     TK|2147483647|00033|      B2|         Air|     Pleasure|2016-07-10|  2016-08-26|      47|  2016-07-10|  

In [26]:
dfs_visatype1.printSchema()
dfs_ac1.printSchema()
dfs_USPoE1.printSchema()
dfs_uszips1.printSchema()
dfs_i94countrycode1.printSchema()
dfs_uscd1.printSchema()
dfs_usdp.printSchema()
dfs_usdr.printSchema()
dfs_temper1.printSchema()
dfs_wc1.printSchema()

dfs_alc.printSchema()
dfs_visapost.printSchema()
dfs_alpha2countrycode.printSchema()
dfs_iap.printSchema()
dfs_USstatecode.printSchema()

dfs_ids1.printSchema()

root
 |-- visatype: string (nullable = true)
 |-- description: string (nullable = true)

root
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- state: string (nullable = true)
 |-- facilities: string (nullable = true)

root
 |-- code: string (nullable = true)
 |-- citystate: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)

root
 |-- zip: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state_id: string (nullable

In [27]:
dfs_ac1.show(5)

+--------------+--------------------+------------+---------+-----------+----------+-------------+---------+----------+--------------------+------------------+-----------------+-----+----------+
|          type|                name|elevation_ft|continent|iso_country|iso_region| municipality|iata_code|local_code|         coordinates|         longitude|         latitude|state|facilities|
+--------------+--------------------+------------+---------+-----------+----------+-------------+---------+----------+--------------------+------------------+-----------------+-----+----------+
| large_airport|Albuquerque Inter...|      5355.0|     null|         US|     US-NM|  Albuquerque|      ABQ|       ABQ|-106.609001, 35.0...|       -106.609001|        35.040199|   NM|        CI|
|medium_airport|Atlantic City Int...|        75.0|     null|         US|     US-NJ|Atlantic City|      ACY|       ACY|-74.5772018432617...|-74.57720184326172|39.45759963989258|   NJ|        CI|
|medium_airport|Akron Fulton I

In [28]:
# close spark sessoion
spark.stop()

#### Reducing the inputs dataset size for project submission
##### Weather dataset is 499MB, so filtering it to 3 countries to reduce size

In [11]:
# World Temperature Data
#df_temper = pd.read_csv('inputs/world_temperature.csv')

In [12]:
# df_temper1  = df_temper.copy()
# print('Total Rows = ',df_temper1.shape[0])
# print('Total Countries = ',df_temper1.Country.unique().shape[0])
# print('Total Cities = ',df_temper1.City.unique().shape[0])
# df_temper1.Country.unique()

Total Rows =  765988
Total Countries =  3
Total Cities =  285


array(['United States', 'Argentina', 'Finland'], dtype=object)

In [9]:
# countries = ['Argentina', 'United States', 'Finland']
# df_temper1 = df_temper1[df_temper1.Country.isin(countries)].copy()
# print('Total Rows = ',df_temper1.shape[0])
# print('Total Countries = ',df_temper1.Country.unique().shape[0])
# print('Total Cities = ',df_temper1.City.unique().shape[0])
# df_temper1.Country.unique()

Total Rows =  765988
Total Countries =  3
Total Cities =  285


array(['United States', 'Argentina', 'Finland'], dtype=object)

In [10]:
# df_temper1.to_csv('inputs/world_temperature.csv', encoding='utf-8', index=False)